In [ ]:
pip install rarfile


In [8]:
import cv2
import numpy as np
import glob

def read_images(folder_path):
    images = []
    for img_path in sorted(glob.glob(folder_path + "/*.pgm")):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        images.append(img)
    return images


def binarize_images(images, threshold=128):
    binarized_images = []
    for img in images:
        _, binarized_img = cv2.threshold(img, threshold, 1, cv2.THRESH_BINARY)
        binarized_images.append(binarized_img)
    return binarized_images

def hamming_distance(img1, img2):
    return np.sum(img1 != img2)

def create_score_matrix(probe_images, gallery_images):
    num_probes = len(probe_images)
    num_galleries = len(gallery_images)
    score_matrix = np.zeros((num_probes, num_galleries))

    for i, probe_image in enumerate(probe_images):
        for j, gallery_image in enumerate(gallery_images):
            score_matrix[i, j] = hamming_distance(probe_image, gallery_image)

    return score_matrix


gallery_folder = "/users/vijay/downloads/GallerySet"
probe_folder = "/users/vijay/downloads/ProbeSet"

gallery_images = read_images(gallery_folder)
probe_images = read_images(probe_folder)

binarized_gallery_images = binarize_images(gallery_images)
binarized_probe_images = binarize_images(probe_images)

score_matrix_A = generate_score_matrix(binarized_probe_images, binarized_gallery_images)

print(score_matrix_A[0:9, 0:9])


[[ 366.  768.  711.  693.  672.  789.  821.  760.  848.]
 [ 677.  195.  776.  738.  873.  594.  650.  753.  711.]
 [ 659.  765.  310.  608.  767.  830.  790.  785.  697.]
 [ 640.  888.  605.  295.  588.  677.  893.  514.  616.]
 [ 662.  974.  655.  507.  164.  649.  759.  584.  690.]
 [ 660.  802.  711.  483.  540.  493.  789.  606.  704.]
 [ 542.  686.  717.  537.  630.  567.  605.  614.  708.]
 [ 583.  857.  746.  468.  563.  688.  826.  249.  743.]
 [ 827.  815.  650.  526.  765.  796. 1062.  701.  399.]]


In [9]:
gallery_images = read_images(gallery_folder)
probe_images = read_images(probe_folder)

print("Number of gallery images:", len(gallery_images))
print("Number of probe images:", len(probe_images))


Number of gallery images: 100
Number of probe images: 100


In [10]:
def decidability_index(genuine_scores, imposter_scores):
    genuine_mean = np.mean(genuine_scores)
    imposter_mean = np.mean(imposter_scores)
    genuine_std = np.std(genuine_scores)
    imposter_std = np.std(imposter_scores)

    d_prime = (np.sqrt(2) * abs(genuine_mean - imposter_mean)) / np.sqrt(genuine_std**2 + imposter_std**2)
    return d_prime

genuine_scores, imposter_scores = separate_scores(score_matrix_A)
d_prime = decidability_index(genuine_scores, imposter_scores)

print("Decidability index (d'): {:.2f}".format(d_prime))


Decidability index (d'): 2.91
